In [20]:
import torch
import torchvision
import numpy as np
import random
import os

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split

label_df = pd.read_csv("./data/train.csv")
train_label_df, valid_label_df = train_test_split(label_df, test_size=0.1, stratify=label_df["has_cactus"], random_state=42)

submission_df = pd.read_csv("./data/sample_submission.csv")

# Data Augmentation

In [32]:
from torchvision import transforms

# 훈련 용 데이터 변환기
transform_train = transforms.Compose([
    transforms.ToTensor(),
    # 원본 이미지에 대칭이되는 32 두께의 이미지를 8개 방향으로 추가 (원본의 3배가 됨)
    # 선인장이 더 추가된 꼴
    transforms.Pad(32, padding_mode="symmetric"),
    # 무작위로 좌우 대칭 변환
    transforms.RandomHorizontalFlip(),
    # 무작위로 상하 대칭 변환
    transforms.RandomVerticalFlip(),
    # 데이터의 지정한 평균과 분산에 맞게 정규화 : 평균(R, G, B), 분산(R, G, B)
    transforms.Normalize((0.485, 0.456, 0.406), # <- ImageNet에서 나온 결과
                         (0.229, 0.224, 0.225))
])

# 검증 및 테스트 용 데이터 변환기
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(32, padding_mode="symmetric"),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

In [33]:
import cv2
import torch.utils

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, label_df, image_dir, transform):
        super(ImageDataset, self).__init__()

        self.label_df = label_df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.label_df)
    
    def __getitem__(self, idx):
        image_file_name = self.label_df.iloc[idx, 0] # id: 파일명
        image_file_path = f"{self.image_dir}/{image_file_name}"
        image = cv2.imread(image_file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        
        label = self.label_df.iloc[idx, 1] # has_cactus

        return image, label

train_dataset = ImageDataset(label_df=train_label_df, image_dir="./data/train", transform=transform_train)
valid_dataset = ImageDataset(label_df=valid_label_df, image_dir="./data/train", transform=transform_test)

train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)

# Model

In [24]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32,
                                            kernel_size=3, padding=2),
                                    nn.BatchNorm2d(32), # parameter: 인풋 채널 수
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer_2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                               kernel_size=3, padding=2),
                                     nn.BatchNorm2d(64),
                                     nn.LeakyReLU(),
                                     nn.MaxPool2d(kernel_size=2))
        
        self.layer_3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128,
                                               kernel_size=3, padding=2),
                                     nn.BatchNorm2d(128),
                                     nn.LeakyReLU(),
                                     nn.MaxPool2d(kernel_size=2))

        self.layer_4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256,
                                               kernel_size=3, padding=2),
                                     nn.BatchNorm2d(256),
                                     nn.LeakyReLU(),
                                     nn.MaxPool2d(kernel_size=2))

        self.layer_5 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512,
                                               kernel_size=3, padding=2),
                                     nn.BatchNorm2d(512),
                                     nn.LeakyReLU(),
                                     nn.MaxPool2d(kernel_size=2))
        
        self.avg_pool = nn.AvgPool2d(kernel_size=4)

        # FC
        self.fc_1 = nn.Linear(in_features=512*1*1, out_features=64)
        
        self.fc_2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, x):
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512*1*1)
        x = self.fc_1(x)
        x = self.fc_2(x)
        return x

# Train

In [25]:
model = Model()
criterion = nn.CrossEntropyLoss()
# batch 크기가 작으면 lr도 작게
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00006)

# epochs = 70
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0

    for images, labels in train_data_loader:
        optimizer.zero_grad()
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        # backward
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}] - loss: {epoch_loss/len(train_data_loader):.4f}")

Epoch [1/10] - loss: 0.1128
Epoch [2/10] - loss: 0.0546
Epoch [3/10] - loss: 0.0393
Epoch [4/10] - loss: 0.0317
Epoch [5/10] - loss: 0.0280
Epoch [6/10] - loss: 0.0218
Epoch [7/10] - loss: 0.0215
Epoch [8/10] - loss: 0.0191
Epoch [9/10] - loss: 0.0182
Epoch [10/10] - loss: 0.0156


# Evaluate

In [34]:
from sklearn.metrics import roc_auc_score

true_list = [] # 실제값
preds_list = [] # 예측값

model.eval() # 평가 상태로 변경

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in valid_data_loader:
        outputs = model(images)
        preds = torch.softmax(outputs, dim=1)[:, 1] # 선인장이 포함될 확률을 가져온다
        true = labels

        preds_list.extend(preds)
        true_list.extend(true)

print(f"ROC AUC: {roc_auc_score(true_list, preds_list):.4f}")

ROC AUC: 0.9997


# Submission

In [36]:
test_dataset = ImageDataset(label_df=submission_df, image_dir="./data/test", transform=transform_test)
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

model.eval()
preds = []

with torch.no_grad():
    for images, _label in test_data_loader:
        outputs = model(images)
        preds_part = torch.softmax(outputs, dim=1)[:, 1].tolist()
        preds.extend(preds_part)

In [39]:
submission_df["has_cactus"] = preds
submission_df.to_csv("./data/deeper_cnn_submission.csv")